In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile"
def main():
    def initialize_driver():
        driver = webdriver.Chrome()
        driver.maximize_window()
        return driver
    
    def load_page(driver, url):
        driver.get(url)
        time.sleep(5)  # Wait for the page to load
    
    # Function to scrape bus routes
    def scrape_bus_routes(driver):
        route_elements = driver.find_elements(By.CLASS_NAME, 'route')
        bus_routes_link = [route.get_attribute('href') for route in route_elements]
        bus_routes_name = [route.text.strip() for route in route_elements]
        return bus_routes_link, bus_routes_name
    
    # Function to scrape bus details
    def scrape_bus_details(driver, url, route_name):
        try:
            driver.get(url)
            time.sleep(5)  # Allow the page to load
            
            # Click the "View Buses" button if it exists
            try:
                view_buses_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CLASS_NAME, "button"))
                )
                driver.execute_script("arguments[0].click();", view_buses_button)
                time.sleep(5)  # Wait for buses to load
                
                # Scroll down to load all bus items
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(5)  # Wait for the page to load more content
    
                # Find bus item details
                bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
                bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
                departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
                duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
                reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
                star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
                price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")
    
                # Use XPath to handle both seat availability classes
                seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")
    
                bus_details = []
                for i in range(len(bus_name_elements)):
                    bus_detail = {
                        "Route_Name": route_name,
                        "Route_Link": url,
                        "Bus_Name": bus_name_elements[i].text,
                        "Bus_Type": bus_type_elements[i].text,
                        "Departing_Time": departing_time_elements[i].text,
                        "Duration": duration_elements[i].text,
                        "Reaching_Time": reaching_time_elements[i].text,
                        "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                        "Price": price_elements[i].text,
                        "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                    }
                    bus_details.append(bus_detail)
                return bus_details
            
            except Exception as e:
                print(f"Error occurred while scraping bus details for {url}: {str(e)}")
                return []
    
        except Exception as e:
            print(f"Error occurred while accessing {url}: {str(e)}")
            return []
    
    # List to hold all bus details
    all_bus_details = []
    
    # Function to scrape all pages
    def scrape_all_pages():
        for page in range(1, 6):  # There are 5 pages
            try:
                driver = initialize_driver()
                load_page(driver, URL)
                
                if page > 1:
                    pagination_tab = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                    )
                    driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                    driver.execute_script("arguments[0].click();", pagination_tab)
                    time.sleep(5)  # Wait for the page to load
                
                all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
                # Iterate over each bus route link and scrape the details
                for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                    bus_details = scrape_bus_details(driver, link, name)
                    if bus_details:
                        all_bus_details.extend(bus_details)
                
    
            except Exception as e:
                print(f"Error occurred while accessing page {page}: {str(e)}")
    
    driver = initialize_driver()
    
    # Scrape routes and details from all pages
    scrape_all_pages()
    
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(all_bus_details)
    return df,driver
df, driver = main()
# Save the DataFrame to a CSV file
df.to_csv('ap_bus_details.csv', index=False)

# Close the driver
driver.quit()
            

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/hyderabad-to-vijayawada: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7033D6CB5+28821]
	(No symbol) [0x00007FF703343840]
	(No symbol) [0x00007FF7031E578A]
	(No symbol) [0x00007FF7032391BE]
	(No symbol) [0x00007FF7032394AC]
	(No symbol) [0x00007FF703282647]
	(No symbol) [0x00007FF70325F33F]
	(No symbol) [0x00007FF70327F412]
	(No symbol) [0x00007FF70325F0A3]
	(No symbol) [0x00007FF70322A778]
	(No symbol) [0x00007FF70322B8E1]
	GetHandleVerifier [0x00007FF70370FCAD+3408013]
	GetHandleVerifier [0x00007FF70372741F+3504127]
	GetHandleVerifier [0x00007FF70371B5FD+3455453]
	GetHandleVerifier [0x00007FF70349BDBB+835995]
	(No symbol) [0x00007FF70334EB5F]
	(No symbol) [0x00007FF70334A814]
	(No symbol) [0x00007FF70334A9AD]
	(No symbol) [0x00007FF70333A199]
	BaseThreadInitThunk [0x00007FF8A3B5259D+29]
	RtlUserThreadStart [0x00007FF8A444AF38+40]



In [13]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 23879,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",13:50,05h 00m,18:50,3.4,INR 436,33 Seats available
1,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 3571,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),14:00,06h 04m,20:04,3.9,INR 683,41 Seats available
2,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 3562,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",14:00,06h 35m,20:35,2.1,INR 448,33 Seats available
3,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 3583,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),14:30,06h 00m,20:30,4.1,INR 615,35 Seats available
4,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 3549,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",14:30,06h 30m,21:00,3.5,INR 448,33 Seats available
...,...,...,...,...,...,...,...,...,...,...
2492,Hyderabad to Nandyal,https://www.redbus.in/bus-tickets/hyderabad-to...,NSK Travels,A/C Sleeper (2+1),18:30,06h 00m,00:30,4.0,1169,15 Seats available
2493,Hyderabad to Nandyal,https://www.redbus.in/bus-tickets/hyderabad-to...,Matangi Travels,Non A/C Seater / Sleeper (2+1),22:10,06h 15m,04:25,4.5,347,24 Seats available
2494,Hyderabad to Nandyal,https://www.redbus.in/bus-tickets/hyderabad-to...,Vikram Travels,A/C Sleeper (2+1),20:20,07h 10m,03:30,4.5,INR 999,23 Seats available
2495,Hyderabad to Nandyal,https://www.redbus.in/bus-tickets/hyderabad-to...,Moksha Travels,A/C Sleeper (2+1),17:00,07h 10m,00:10,4.4,INR 999,13 Seats available


In [14]:
# Assam Statae
# URL of the website
URL = "https://www.redbus.in/online-booking/astc/?utm_source=rtchometile"
df,driver = main()
# Save the DataFrame to a CSV file
df.to_csv('Assam_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/goalpara-to-guwahati: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7033D6CB5+28821]
	(No symbol) [0x00007FF703343840]
	(No symbol) [0x00007FF7031E578A]
	(No symbol) [0x00007FF7032391BE]
	(No symbol) [0x00007FF7032394AC]
	(No symbol) [0x00007FF703282647]
	(No symbol) [0x00007FF70325F33F]
	(No symbol) [0x00007FF70327F412]
	(No symbol) [0x00007FF70325F0A3]
	(No symbol) [0x00007FF70322A778]
	(No symbol) [0x00007FF70322B8E1]
	GetHandleVerifier [0x00007FF70370FCAD+3408013]
	GetHandleVerifier [0x00007FF70372741F+3504127]
	GetHandleVerifier [0x00007FF70371B5FD+3455453]
	GetHandleVerifier [0x00007FF70349BDBB+835995]
	(No symbol) [0x00007FF70334EB5F]
	(No symbol) [0x00007FF70334A814]
	(No symbol) [0x00007FF70334A9AD]
	(No symbol) [0x00007FF70333A199]
	BaseThreadInitThunk [0x00007FF8A3B5259D+29]
	RtlUserThreadStart [0x00007FF8A444AF38+40]

Error occurred while scraping bus details for https://www.redbus.in/bus-t

In [15]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Assam State Transport Corporation (ASTC) - 139847,Volvo AC Seater 2+2,16:15,04h 15m,20:30,4.1,INR 298,6 Seats available
1,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Chartered Bus - ASTC,Volvo Multi-Axle I-Shift B11R Semi Sleeper (2+2),13:00,04h 00m,17:00,4.5,INR 397.08,8 Seats available
2,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Chartered Bus - ASTC,Volvo AC Seater Pushback 2+2,15:30,03h 45m,19:15,4.5,INR 397.08,1 Seat available
3,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),21:45,05h 30m,03:15,3.5,450,3 Seats available
4,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,PANKAJ TRAVELS,AC Seater (2+2),13:10,04h 40m,17:50,2.4,315,32 Seats available
...,...,...,...,...,...,...,...,...,...,...
242,North Lakhimpur to Nagaon (Assam),https://www.redbus.in/bus-tickets/north-lakhim...,Yatra Travels,NON A/C Seater (2+1),20:40,06h 20m,03:00,4.1,450,27 Seats available
243,North Lakhimpur to Nagaon (Assam),https://www.redbus.in/bus-tickets/north-lakhim...,Yatra Travels,A/C Seater / Sleeper (2+1),21:00,06h 30m,03:30,3.9,500,30 Seats available
244,North Lakhimpur to Nagaon (Assam),https://www.redbus.in/bus-tickets/north-lakhim...,Island Travels (Under ASTC),NON AC Seater / Sleeper 2+1,20:40,06h 20m,03:00,3.5,601,40 Seats available
245,North Lakhimpur to Nagaon (Assam),https://www.redbus.in/bus-tickets/north-lakhim...,DESTINY TRAVELS,NON AC Seater / Sleeper 2+1,18:30,07h 00m,01:30,1.5,INR 498.75,35 Seats available


In [16]:
# Bihar
# URL of the website
URL = "https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile"
df,driver = main()
# Save the DataFrame to a CSV file
df.to_csv('Bihar_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/patna-to-kathmandu: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7033D6CB5+28821]
	(No symbol) [0x00007FF703343840]
	(No symbol) [0x00007FF7031E578A]
	(No symbol) [0x00007FF7032391BE]
	(No symbol) [0x00007FF7032394AC]
	(No symbol) [0x00007FF703282647]
	(No symbol) [0x00007FF70325F33F]
	(No symbol) [0x00007FF70327F412]
	(No symbol) [0x00007FF70325F0A3]
	(No symbol) [0x00007FF70322A778]
	(No symbol) [0x00007FF70322B8E1]
	GetHandleVerifier [0x00007FF70370FCAD+3408013]
	GetHandleVerifier [0x00007FF70372741F+3504127]
	GetHandleVerifier [0x00007FF70371B5FD+3455453]
	GetHandleVerifier [0x00007FF70349BDBB+835995]
	(No symbol) [0x00007FF70334EB5F]
	(No symbol) [0x00007FF70334A814]
	(No symbol) [0x00007FF70334A9AD]
	(No symbol) [0x00007FF70333A199]
	BaseThreadInitThunk [0x00007FF8A3B5259D+29]
	RtlUserThreadStart [0x00007FF8A444AF38+40]

Error occurred while scraping bus details for https://www.redbus.in/bus-tic

In [17]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...,Jai Hanuman Travels,A/C Seater / Sleeper (2+2),22:20,06h 40m,05:00,3.6,495,54 Seats available
1,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...,Jai Mata Di Tour and Travels,A/C Seater / Sleeper (2+2),22:40,05h 20m,04:00,3.4,INR 450,25 Seats available
2,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...,Vivek Travels,A/C Seater (2+2),13:25,07h 00m,20:25,1.0,INR 450,45 Seats available
3,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...,Vivek Travels,A/C Seater / Sleeper (2+2),13:30,07h 00m,20:30,1.0,INR 400,72 Seats available
4,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...,Jai Mata Di Tour and Travels,A/C Seater / Sleeper (2+2),20:28,05h 32m,02:00,2.0,INR 500,6 Seats available
...,...,...,...,...,...,...,...,...,...,...
366,Patna (Bihar) to Ramgarh (Jharkhand),https://www.redbus.in/bus-tickets/patna-to-ram...,Tejas Maa Bhawani,Non AC Seater (2+2),21:30,06h 20m,03:50,2.0,INR 2000,41 Seats available
367,Patna (Bihar) to Ramgarh (Jharkhand),https://www.redbus.in/bus-tickets/patna-to-ram...,Sri Krishna Rath,A/C Seater/Sleeper (2+1),20:20,10h 10m,06:30,1.6,1313,40 Seats available
368,Patna (Bihar) to Ramgarh (Jharkhand),https://www.redbus.in/bus-tickets/patna-to-ram...,Babuan Motors,A/C Seater / Sleeper (2+1),19:00,07h 00m,02:00,1.3,INR 500,19 Seats available
369,Patna (Bihar) to Ramgarh (Jharkhand),https://www.redbus.in/bus-tickets/patna-to-ram...,Sri Krishna Rath,A/C Seater / Sleeper (2+1),22:15,07h 00m,05:15,0,INR 550,15 Seats available


In [18]:
# HimachalPradesh
# URL of the website
URL = "https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile"
df,driver = main()
# Save the DataFrame to a CSV file
df.to_csv('HimachalPradesh_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/delhi-to-shimla: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7033D6CB5+28821]
	(No symbol) [0x00007FF703343840]
	(No symbol) [0x00007FF7031E578A]
	(No symbol) [0x00007FF7032391BE]
	(No symbol) [0x00007FF7032394AC]
	(No symbol) [0x00007FF703282647]
	(No symbol) [0x00007FF70325F33F]
	(No symbol) [0x00007FF70327F412]
	(No symbol) [0x00007FF70325F0A3]
	(No symbol) [0x00007FF70322A778]
	(No symbol) [0x00007FF70322B8E1]
	GetHandleVerifier [0x00007FF70370FCAD+3408013]
	GetHandleVerifier [0x00007FF70372741F+3504127]
	GetHandleVerifier [0x00007FF70371B5FD+3455453]
	GetHandleVerifier [0x00007FF70349BDBB+835995]
	(No symbol) [0x00007FF70334EB5F]
	(No symbol) [0x00007FF70334A814]
	(No symbol) [0x00007FF70334A9AD]
	(No symbol) [0x00007FF70333A199]
	BaseThreadInitThunk [0x00007FF8A3B5259D+29]
	RtlUserThreadStart [0x00007FF8A444AF38+40]

Error occurred while scraping bus details for https://www.redbus.in/bus-ticket

In [19]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Chandigarh to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/chandigarh-t...,HRTC - 1516,A/C Executive (2+3),15:00,03h 50m,18:50,4.2,INR 340,35 Seats available
1,Chandigarh to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/chandigarh-t...,HRTC - 1468,Ordinary 3+2 Non AC Seater,15:30,05h 50m,21:20,3.9,INR 302,37 Seats available
2,Chandigarh to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/chandigarh-t...,HRTC - 1546,Ordinary 3+2 Non AC Seater,16:25,04h 50m,21:15,2.5,INR 334,26 Seats available
3,Chandigarh to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/chandigarh-t...,HRTC - 9235,A/C Executive (2+3),16:40,05h 20m,22:00,4.7,INR 399,44 Seats available
4,Chandigarh to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/chandigarh-t...,HRTC - 401670,Ordinary 3+2 Non AC Seater,18:00,04h 00m,22:00,3.0,INR 310,31 Seats available
...,...,...,...,...,...,...,...,...,...,...
710,Manali to Shimla,https://www.redbus.in/bus-tickets/manali-to-sh...,HRTC - 710,Himmani Deluxe 2+2 Non AC Seater,18:20,10h 10m,04:30,3.6,INR 641,24 Seats available
711,Manali to Shimla,https://www.redbus.in/bus-tickets/manali-to-sh...,HRTC - 155,Himsuta AC Seater Volvo/Scania 2+2,21:00,09h 10m,06:10,3.9,INR 963,18 Seats available
712,Manali to Shimla,https://www.redbus.in/bus-tickets/manali-to-sh...,HRTC - 718,Ordinary 3+2 Non AC Seater,21:30,09h 05m,06:35,4.1,INR 583,37 Seats available
713,Baddi (Himachal Pradesh) to Delhi,https://www.redbus.in/bus-tickets/baddi-himach...,HRTC - 6709,Ordinary 3+2 Non AC Seater,19:30,07h 10m,02:40,3.8,INR 325,30 Seats available


In [20]:
# JammuKashmir
# URL of the website
URL = "https://www.redbus.in/online-booking/jksrtc"
df,driver = main()
# Save the DataFrame to a CSV file
df.to_csv('JammuKashmir_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while accessing page 2: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7033D6CB5+28821]
	(No symbol) [0x00007FF703343840]
	(No symbol) [0x00007FF7031E578A]
	(No symbol) [0x00007FF7032391BE]
	(No symbol) [0x00007FF7032394AC]
	(No symbol) [0x00007FF703282647]
	(No symbol) [0x00007FF70325F33F]
	(No symbol) [0x00007FF70327F412]
	(No symbol) [0x00007FF70325F0A3]
	(No symbol) [0x00007FF70322A778]
	(No symbol) [0x00007FF70322B8E1]
	GetHandleVerifier [0x00007FF70370FCAD+3408013]
	GetHandleVerifier [0x00007FF70372741F+3504127]
	GetHandleVerifier [0x00007FF70371B5FD+3455453]
	GetHandleVerifier [0x00007FF70349BDBB+835995]
	(No symbol) [0x00007FF70334EB5F]
	(No symbol) [0x00007FF70334A814]
	(No symbol) [0x00007FF70334A9AD]
	(No symbol) [0x00007FF70333A199]
	BaseThreadInitThunk [0x00007FF8A3B5259D+29]
	RtlUserThreadStart [0x00007FF8A444AF38+40]

Error occurred while accessing page 3: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7033D6CB5+28821]
	(No symbol) [0x00007FF703343

In [21]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Delhi to Srinagar,https://www.redbus.in/bus-tickets/delhi-to-sri...,Zingbus Plus,A/C Sleeper (2+1),21:35,19h 30m,17:05,4.7,2138,16 Seats available
1,Delhi to Srinagar,https://www.redbus.in/bus-tickets/delhi-to-sri...,Zingbus Plus,Bharat Benz A/C Seater /Sleeper (2+1),20:00,19h 20m,15:20,3.9,2137,14 Seats available
2,Delhi to Srinagar,https://www.redbus.in/bus-tickets/delhi-to-sri...,RAO TRAVELS INDIA PVT LTD.,Mercedes Benz A/C (2+2),16:00,19h 30m,11:30,2.2,INR 3000,48 Seats available
3,Jammu (j and k) to Katra (jammu and kashmir),https://www.redbus.in/bus-tickets/jammu-to-katra,Bhawani Travels,A/C Seater / Sleeper (2+2),19:50,01h 40m,21:30,3.8,400,36 Seats available
4,Jammu (j and k) to Katra (jammu and kashmir),https://www.redbus.in/bus-tickets/jammu-to-katra,Bhawani Travels,A/C Seater / Sleeper (2+2),20:00,01h 45m,21:45,3.8,400,28 Seats available
5,Jammu (j and k) to Katra (jammu and kashmir),https://www.redbus.in/bus-tickets/jammu-to-katra,Namdev Travels,NON A/C Seater (2+2),16:00,02h 00m,18:00,0,INR 100,34 Seats available
6,Jammu (j and k) to Katra (jammu and kashmir),https://www.redbus.in/bus-tickets/jammu-to-katra,PK VAID TRAVELS,NON A/C Seater (2+3),17:00,01h 30m,18:30,0,INR 150,36 Seats available
7,Katra (jammu and kashmir) to Jammu (j and k),https://www.redbus.in/bus-tickets/katra-to-jammu,Shiv Mohan Travels,AC Sleeper (2+1),17:00,01h 30m,18:30,3.1,INR 619.05,36 Seats available
8,Katra (jammu and kashmir) to Jammu (j and k),https://www.redbus.in/bus-tickets/katra-to-jammu,FlixBus,Bharat Benz A/C Sleeper (2+1),18:30,01h 30m,20:00,4.1,INR 88.39,22 Seats available
9,Katra (jammu and kashmir) to Jammu (j and k),https://www.redbus.in/bus-tickets/katra-to-jammu,Zingbus Plus,Bharat Benz A/C Seater /Sleeper (2+1),18:45,01h 25m,20:10,4.5,INR 103,42 Seats available


In [22]:
# Kadamba
# URL of the website
URL = "https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile"
df,driver = main()
# Save the DataFrame to a CSV file
df.to_csv('Kadamba_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/calangute-goa-to-mopa-airport: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7033D6CB5+28821]
	(No symbol) [0x00007FF703343840]
	(No symbol) [0x00007FF7031E578A]
	(No symbol) [0x00007FF7032391BE]
	(No symbol) [0x00007FF7032394AC]
	(No symbol) [0x00007FF703282647]
	(No symbol) [0x00007FF70325F33F]
	(No symbol) [0x00007FF70327F412]
	(No symbol) [0x00007FF70325F0A3]
	(No symbol) [0x00007FF70322A778]
	(No symbol) [0x00007FF70322B8E1]
	GetHandleVerifier [0x00007FF70370FCAD+3408013]
	GetHandleVerifier [0x00007FF70372741F+3504127]
	GetHandleVerifier [0x00007FF70371B5FD+3455453]
	GetHandleVerifier [0x00007FF70349BDBB+835995]
	(No symbol) [0x00007FF70334EB5F]
	(No symbol) [0x00007FF70334A814]
	(No symbol) [0x00007FF70334A9AD]
	(No symbol) [0x00007FF70333A199]
	BaseThreadInitThunk [0x00007FF8A3B5259D+29]
	RtlUserThreadStart [0x00007FF8A444AF38+40]

Error occurred while scraping bus details for https://www.redbus

In [23]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Kadamba Transport Corporation Limited (KTCL) -...,Non AC Seater 2+2,19:00,10h 00m,05:00,3.6,INR 600,16 Seats available
1,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,IntrCity SmartBus,AC Sleeper (2+1),22:15,10h 30m,08:45,4.1,INR 1030,6 Seats available
2,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Gogte Travels,A/C Sleeper (2+1),20:15,10h 45m,07:00,4.3,INR 1309,7 Seats available
3,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,AdIntrCity SmartBus,Bharat Benz A/C Sleeper (2+1),20:25,10h 45m,07:10,4.2,INR 1739,12 Seats available
4,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Ashray Travels,Non A/C Seater / Sleeper (2+1),21:20,11h 30m,08:50,4.3,INR 650,12 Seats available
...,...,...,...,...,...,...,...,...,...,...
468,Goa to Shirdi,https://www.redbus.in/bus-tickets/goa-to-shirdi,PSR Travels Goa,A/C Sleeper (2+1),19:15,14h 45m,10:00,3.4,INR 1399,9 Seats available
469,Goa to Shirdi,https://www.redbus.in/bus-tickets/goa-to-shirdi,Humsafar Travels,Bharat Benz A/C Sleeper (2+1),19:00,16h 00m,11:00,2.3,INR 1450,17 Seats available
470,Mopa Airport to Panaji,https://www.redbus.in/bus-tickets/mopa-airport...,Kadamba Transport Corporation Limited (KTCL) -...,A/C Seater (2+3),16:00,00h 55m,16:55,3.6,INR 200,47 Seats available
471,Belagavi to Saquelim,https://www.redbus.in/bus-tickets/belagavi-to-...,Kadamba Transport Corporation Limited (KTCL) -...,Non AC Seater 2+2,16:00,02h 15m,18:15,3.6,INR 140,3 Seats available


In [24]:
# Kerala
# URL of the website
URL = "https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile"
df,driver = main()
# Save the DataFrame to a CSV file
df.to_csv('Kerala_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while accessing page 3: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7033D6CB5+28821]
	(No symbol) [0x00007FF703343840]
	(No symbol) [0x00007FF7031E578A]
	(No symbol) [0x00007FF7032391BE]
	(No symbol) [0x00007FF7032394AC]
	(No symbol) [0x00007FF703282647]
	(No symbol) [0x00007FF70325F33F]
	(No symbol) [0x00007FF70327F412]
	(No symbol) [0x00007FF70325F0A3]
	(No symbol) [0x00007FF70322A778]
	(No symbol) [0x00007FF70322B8E1]
	GetHandleVerifier [0x00007FF70370FCAD+3408013]
	GetHandleVerifier [0x00007FF70372741F+3504127]
	GetHandleVerifier [0x00007FF70371B5FD+3455453]
	GetHandleVerifier [0x00007FF70349BDBB+835995]
	(No symbol) [0x00007FF70334EB5F]
	(No symbol) [0x00007FF70334A814]
	(No symbol) [0x00007FF70334A9AD]
	(No symbol) [0x00007FF70333A199]
	BaseThreadInitThunk [0x00007FF8A3B5259D+29]
	RtlUserThreadStart [0x00007FF8A444AF38+40]

Error occurred while accessing page 4: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7033D6CB5+28821]
	(No symbol) [0x00007FF703343

In [25]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 235,Swift Deluxe Non AC Air Bus (2+2),15:00,10h 00m,01:00,3.6,INR 620,5 Seats available
1,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 97,Swift Deluxe Non AC Air Bus (2+2),16:00,08h 45m,00:45,3.5,INR 567,11 Seats available
2,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 380,Swift Deluxe Non AC Air Bus (2+2),16:15,08h 45m,01:00,4.6,INR 567,4 Seats available
3,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 553,Super Express Non AC Seater Air Bus (2+2),16:45,09h 30m,02:15,3.5,INR 640,5 Seats available
4,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 51,Swift Deluxe Non AC Air Bus (2+2),19:00,09h 30m,04:30,3.9,INR 640,20 Seats available
...,...,...,...,...,...,...,...,...,...,...
692,Kozhikode to Aluva,https://www.redbus.in/bus-tickets/kozhikode-to...,BLM Transports,A/C Sleeper (2+1),22:00,04h 15m,02:15,3.0,855,9 Seats available
693,Kozhikode to Aluva,https://www.redbus.in/bus-tickets/kozhikode-to...,EMERALD TRAVELS,A/C Sleeper (2+1),22:10,03h 15m,01:25,4.0,INR 1000,20 Seats available
694,Kozhikode to Aluva,https://www.redbus.in/bus-tickets/kozhikode-to...,Madhavi Travels,A/C Sleeper (2+1),22:20,03h 55m,02:15,3.8,999,30 Seats available
695,Kozhikode to Aluva,https://www.redbus.in/bus-tickets/kozhikode-to...,LIYAS TOURS,Bharat Benz A/C Sleeper (2+1),23:05,03h 20m,02:25,3.3,849,21 Seats available


In [26]:
# NorthBengal
# URL of the website
URL = "https://www.redbus.in/travels/nbstc"
df,driver = main()
# Save the DataFrame to a CSV file
df.to_csv('NorthBengal_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/siliguri-to-darjeeling: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7033D6CB5+28821]
	(No symbol) [0x00007FF703343840]
	(No symbol) [0x00007FF7031E578A]
	(No symbol) [0x00007FF7032391BE]
	(No symbol) [0x00007FF7032394AC]
	(No symbol) [0x00007FF703282647]
	(No symbol) [0x00007FF70325F33F]
	(No symbol) [0x00007FF70327F412]
	(No symbol) [0x00007FF70325F0A3]
	(No symbol) [0x00007FF70322A778]
	(No symbol) [0x00007FF70322B8E1]
	GetHandleVerifier [0x00007FF70370FCAD+3408013]
	GetHandleVerifier [0x00007FF70372741F+3504127]
	GetHandleVerifier [0x00007FF70371B5FD+3455453]
	GetHandleVerifier [0x00007FF70349BDBB+835995]
	(No symbol) [0x00007FF70334EB5F]
	(No symbol) [0x00007FF70334A814]
	(No symbol) [0x00007FF70334A9AD]
	(No symbol) [0x00007FF70333A199]
	BaseThreadInitThunk [0x00007FF8A3B5259D+29]
	RtlUserThreadStart [0x00007FF8A444AF38+40]

Error occurred while scraping bus details for https://www.redbus.in/bus

In [27]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Kolkata to Siliguri,https://www.redbus.in/bus-tickets/kolkata-to-s...,NBSTC-KOL-COB-E2-3:00 pm - 171781,Non AC Seater (2+3),15:00,13h 20m,04:20,3.4,INR 450,31 Seats available
1,Kolkata to Siliguri,https://www.redbus.in/bus-tickets/kolkata-to-s...,NBSTC-KOL-COB-R1-17:00 - 156429,Non AC Seater 2+2,17:00,12h 00m,05:00,3.4,INR 486,3 Seats available
2,Kolkata to Siliguri,https://www.redbus.in/bus-tickets/kolkata-to-s...,NBSTC-KOL-COB-AC 19:30 - 176646,AC Seater (2+2),19:30,11h 50m,07:20,3.0,INR 800,28 Seats available
3,Kolkata to Siliguri,https://www.redbus.in/bus-tickets/kolkata-to-s...,SBSTC-KOLKATA - SILIGURI - VOLVO - 20:05 (BEL ...,Volvo AC Seater (2+2),20:05,13h 00m,09:05,3.9,INR 1210,50 Seats available
4,Kolkata to Siliguri,https://www.redbus.in/bus-tickets/kolkata-to-s...,Easy Ride,Volvo 9600 Multi-Axle A/C Sleeper (2+1),18:30,12h 55m,07:25,4.7,INR 1500,21 Seats available
...,...,...,...,...,...,...,...,...,...,...
369,Kolkata to Kishanganj,https://www.redbus.in/bus-tickets/kolkata-to-k...,Lokenath Bus Service,Volvo 9600 Multi-Axle A/C Sleeper (2+1),18:45,11h 30m,06:15,4.5,INR 1500,14 Seats available
370,Kolkata to Kishanganj,https://www.redbus.in/bus-tickets/kolkata-to-k...,Maa Durga Bus Service,NON AC Seater / Sleeper 2+1,19:00,13h 00m,08:00,2.8,INR 600,13 Seats available
371,Kolkata to Kishanganj,https://www.redbus.in/bus-tickets/kolkata-to-k...,Lokenath (Maa Tara),NON A/C Seater / Sleeper (2+2),15:45,13h 00m,04:45,2.7,540,12 Seats available
372,Siliguri to Barasat (West Bengal),https://www.redbus.in/bus-tickets/siliguri-to-...,NBSTC-SLG-KOL-R1-18:30 - 156410,NON A/C Seater Push Back (2+2),18:30,11h 00m,05:30,4.4,INR 466,13 Seats available


In [28]:
# Punjab
# URL of the website
URL = "https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile"
df,driver = main()
# Save the DataFrame to a CSV file
df.to_csv('Punjab_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/delhi-to-ludhiana: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7033D6CB5+28821]
	(No symbol) [0x00007FF703343840]
	(No symbol) [0x00007FF7031E578A]
	(No symbol) [0x00007FF7032391BE]
	(No symbol) [0x00007FF7032394AC]
	(No symbol) [0x00007FF703282647]
	(No symbol) [0x00007FF70325F33F]
	(No symbol) [0x00007FF70327F412]
	(No symbol) [0x00007FF70325F0A3]
	(No symbol) [0x00007FF70322A778]
	(No symbol) [0x00007FF70322B8E1]
	GetHandleVerifier [0x00007FF70370FCAD+3408013]
	GetHandleVerifier [0x00007FF70372741F+3504127]
	GetHandleVerifier [0x00007FF70371B5FD+3455453]
	GetHandleVerifier [0x00007FF70349BDBB+835995]
	(No symbol) [0x00007FF70334EB5F]
	(No symbol) [0x00007FF70334A814]
	(No symbol) [0x00007FF70334A9AD]
	(No symbol) [0x00007FF70333A199]
	BaseThreadInitThunk [0x00007FF8A3B5259D+29]
	RtlUserThreadStart [0x00007FF8A444AF38+40]

Error occurred while scraping bus details for https://www.redbus.in/bus-tick

In [29]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...,PEPSU (Punjab) - 66667,Volvo AC Seater (2+2),17:31,04h 59m,22:30,4.2,INR 711,38 Seats available
1,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...,Indo Canadian Transport Company,Mercedes A/C Semi Sleeper (2+2),23:30,07h 00m,06:30,4.6,INR 3300,43 Seats available
2,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...,Indo Canadian Transport Company,Mercedes A/C Semi Sleeper (2+2),21:00,07h 00m,04:00,4.6,INR 3300,49 Seats available
3,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...,Indo Canadian Transport Company,Volvo A/C Sleeper (2+1),22:00,07h 00m,05:00,4.6,INR 3850,36 Seats available
4,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...,Indo Canadian Transport Company,A/C Semi Sleeper (2+2),19:00,07h 00m,02:00,4.6,INR 3300,40 Seats available
...,...,...,...,...,...,...,...,...,...,...
598,Amritsar to Delhi Airport,https://www.redbus.in/bus-tickets/amritsar-to-...,CALL2TRIP,A/C Seater / Sleeper (2+2),22:25,07h 45m,06:10,2.7,INR 450,18 Seats available
599,Delhi Airport to Kapurthala,https://www.redbus.in/bus-tickets/delhi-airpor...,PEPSU (Punjab) - 67064,Volvo AC Seater (2+2),23:58,10h 00m,09:58,3.7,INR 1226,35 Seats available
600,Amritsar to Patiala,https://www.redbus.in/bus-tickets/amritsar-to-...,PEPSU (Punjab) - 66674,HVAC Seater (2+3),18:10,04h 50m,23:00,3.7,INR 469,54 Seats available
601,Amritsar to Patiala,https://www.redbus.in/bus-tickets/amritsar-to-...,GK Travels,Bharat Benz A/C Seater /Sleeper (2+1),21:00,04h 30m,01:30,3.7,854,39 Seats available


In [30]:
# Rajasthan
# URL of the website
URL = "https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile"
df,driver = main()
# Save the DataFrame to a CSV file
df.to_csv('Rajasthan_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while accessing page 3: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7033D6CB5+28821]
	(No symbol) [0x00007FF703343840]
	(No symbol) [0x00007FF7031E578A]
	(No symbol) [0x00007FF7032391BE]
	(No symbol) [0x00007FF7032394AC]
	(No symbol) [0x00007FF703282647]
	(No symbol) [0x00007FF70325F33F]
	(No symbol) [0x00007FF70327F412]
	(No symbol) [0x00007FF70325F0A3]
	(No symbol) [0x00007FF70322A778]
	(No symbol) [0x00007FF70322B8E1]
	GetHandleVerifier [0x00007FF70370FCAD+3408013]
	GetHandleVerifier [0x00007FF70372741F+3504127]
	GetHandleVerifier [0x00007FF70371B5FD+3455453]
	GetHandleVerifier [0x00007FF70349BDBB+835995]
	(No symbol) [0x00007FF70334EB5F]
	(No symbol) [0x00007FF70334A814]
	(No symbol) [0x00007FF70334A9AD]
	(No symbol) [0x00007FF70333A199]
	BaseThreadInitThunk [0x00007FF8A3B5259D+29]
	RtlUserThreadStart [0x00007FF8A444AF38+40]

Error occurred while accessing page 4: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7033D6CB5+28821]
	(No symbol) [0x00007FF703343

In [31]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 190572,Express Non AC Seater 2+3,16:15,04h 45m,21:00,4.5,INR 229,39 Seats available
1,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 194178,Super Luxury Volvo AC Seater Pushback 2+2,16:30,04h 00m,20:30,4.4,INR 460,32 Seats available
2,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 160785,Super Luxury Volvo AC Seater Pushback 2+2,17:30,04h 00m,21:30,3.5,INR 460,35 Seats available
3,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 190951,Express Non AC Seater 2+3,18:00,04h 30m,22:30,3.9,INR 229,42 Seats available
4,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 193481,Express Non AC Seater 2+3,18:30,04h 30m,23:00,3.9,INR 229,42 Seats available
...,...,...,...,...,...,...,...,...,...,...
762,Udaipur to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/udaipur-to-j...,Gajraj bus service,Bharat Benz A/C Seater /Sleeper (2+1),19:40,08h 50m,04:30,2.2,INR 699,38 Seats available
763,Udaipur to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/udaipur-to-j...,Rishabh Travels Ajay Raj,Bharat Benz A/C Seater /Sleeper (2+1),21:55,07h 10m,05:05,2.4,555,26 Seats available
764,Udaipur to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/udaipur-to-j...,Kalpana Travels Pvt. Ltd.,A/C Seater / Sleeper (2+1),22:15,08h 00m,06:15,2.4,INR 400,35 Seats available
765,Udaipur to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/udaipur-to-j...,Raj travels Jhunjhunu,A/C Seater / Sleeper (2+1),18:00,08h 00m,02:00,1.2,570,37 Seats available


In [32]:
# SouthBengal
# URL of the website
URL = "https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile"
df,driver = main()
# Save the DataFrame to a CSV file
df.to_csv('SouthBengal_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/kolkata-to-contai-kanthi: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7033D6CB5+28821]
	(No symbol) [0x00007FF703343840]
	(No symbol) [0x00007FF7031E578A]
	(No symbol) [0x00007FF7032391BE]
	(No symbol) [0x00007FF7032394AC]
	(No symbol) [0x00007FF703282647]
	(No symbol) [0x00007FF70325F33F]
	(No symbol) [0x00007FF70327F412]
	(No symbol) [0x00007FF70325F0A3]
	(No symbol) [0x00007FF70322A778]
	(No symbol) [0x00007FF70322B8E1]
	GetHandleVerifier [0x00007FF70370FCAD+3408013]
	GetHandleVerifier [0x00007FF70372741F+3504127]
	GetHandleVerifier [0x00007FF70371B5FD+3455453]
	GetHandleVerifier [0x00007FF70349BDBB+835995]
	(No symbol) [0x00007FF70334EB5F]
	(No symbol) [0x00007FF70334A814]
	(No symbol) [0x00007FF70334A9AD]
	(No symbol) [0x00007FF70333A199]
	BaseThreadInitThunk [0x00007FF8A3B5259D+29]
	RtlUserThreadStart [0x00007FF8A444AF38+40]

Error occurred while scraping bus details for https://www.redbus.in/b

In [33]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-DURGAPUR - KARUNAMOYEE - A/C - 15:10 (DG...,AC Seater (2+2),15:10,04h 00m,19:10,3.3,INR 300,31 Seats available
1,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-BANKURA - KARUNAMOYEE - 14:15 (BNK DEPOT...,Non AC Seater (2+3),16:00,04h 00m,20:00,4.0,INR 155,18 Seats available
2,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-ASANSOL - KOLKATA - 15:05 (HWH DEPOT) - 210,Non AC Seater (2+3),16:15,03h 30m,19:45,3.6,INR 150,50 Seats available
3,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-ASANSOL - KARUNAMOYEE - 15:10 (BEL DEPOT...,Non AC Seater (2+3),16:20,04h 00m,20:20,3.4,INR 155,39 Seats available
4,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-ASANSOL - KOLKATA - 15:35 (HOWRAH DEPOT)...,Non AC Seater (2+3),16:45,03h 30m,20:15,3.7,INR 150,52 Seats available
...,...,...,...,...,...,...,...,...,...,...
400,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...,Annesha Travels,NON A/C Seater / Sleeper (2+2),18:00,13h 00m,07:00,2.4,INR 500,30 Seats available
401,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...,Maa Durga Bus Service,NON A/C Seater / Sleeper (2+2),18:00,14h 00m,08:00,2.8,INR 600,5 Seats available
402,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...,Maulik Paribahan,A/C Seater / Sleeper (2+2),18:30,11h 30m,06:00,2.0,INR 1000,44 Seats available
403,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...,RAYAN,A/C Seater / Sleeper (2+1),18:15,12h 00m,06:15,2.1,INR 1500,55 Seats available


In [34]:
# Telangana
# URL of the website
URL = "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile"
df,driver = main()
# Save the DataFrame to a CSV file
df.to_csv('Telangana_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while accessing page 4: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7033D6CB5+28821]
	(No symbol) [0x00007FF703343840]
	(No symbol) [0x00007FF7031E578A]
	(No symbol) [0x00007FF7032391BE]
	(No symbol) [0x00007FF7032394AC]
	(No symbol) [0x00007FF703282647]
	(No symbol) [0x00007FF70325F33F]
	(No symbol) [0x00007FF70327F412]
	(No symbol) [0x00007FF70325F0A3]
	(No symbol) [0x00007FF70322A778]
	(No symbol) [0x00007FF70322B8E1]
	GetHandleVerifier [0x00007FF70370FCAD+3408013]
	GetHandleVerifier [0x00007FF70372741F+3504127]
	GetHandleVerifier [0x00007FF70371B5FD+3455453]
	GetHandleVerifier [0x00007FF70349BDBB+835995]
	(No symbol) [0x00007FF70334EB5F]
	(No symbol) [0x00007FF70334A814]
	(No symbol) [0x00007FF70334A9AD]
	(No symbol) [0x00007FF70333A199]
	BaseThreadInitThunk [0x00007FF8A3B5259D+29]
	RtlUserThreadStart [0x00007FF8A444AF38+40]

Error occurred while accessing page 5: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7033D6CB5+28821]
	(No symbol) [0x00007FF703343

In [35]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TGSRTC - 8665,LAHARI A/C SLEEPER CUM SEATER,15:00,05h 00m,20:00,4.1,INR 389,26 Seats available
1,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TGSRTC - 8969,Super Luxury (Non AC Seater 2+2 Push Back),15:15,04h 15m,19:30,4.1,INR 320,26 Seats available
2,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TGSRTC - 8929,Super Luxury (Non AC Seater 2+2 Push Back),15:30,04h 00m,19:30,3.7,INR 320,28 Seats available
3,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TGSRTC - 8698,RAJDHANI (A.C. Semi Sleeper),15:30,04h 00m,19:30,3.6,INR 389,37 Seats available
4,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TGSRTC - 8623,Rajdhani (AC Semi Sleeper 2+2),16:00,03h 20m,19:20,4.2,INR 389,32 Seats available
...,...,...,...,...,...,...,...,...,...,...
1022,Hyderabad to Kodad,https://www.redbus.in/bus-tickets/hyderabad-to...,Morning Star Travels,A/C Sleeper (2+1),23:10,03h 50m,03:00,2.7,INR 590,27 Seats available
1023,Hyderabad to Kodad,https://www.redbus.in/bus-tickets/hyderabad-to...,Morning Star Travels,A/C Sleeper (2+1),23:10,03h 50m,03:00,2.8,INR 690,30 Seats available
1024,Hyderabad to Kodad,https://www.redbus.in/bus-tickets/hyderabad-to...,NueGo,Electric A/C Seater (2+2),23:15,05h 35m,04:50,2.7,408,24 Seats available
1025,Hyderabad to Kodad,https://www.redbus.in/bus-tickets/hyderabad-to...,Morning Star Travels,A/C Sleeper (2+1),23:15,04h 05m,03:20,2.7,INR 590,23 Seats available


In [36]:
# UttaraPradesh
# URL of the website
URL = "https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile"
df,driver = main()
# Save the DataFrame to a CSV file
df.to_csv('UttaraPradesh_bus_details.csv', index=False)

# Close the driver
driver.quit()

In [37]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Delhi to Bareilly (Uttar Pradesh),https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - HDD0085,Ordinary Non AC Seater 2+3,16:06,08h 19m,00:25,3.3,INR 444,52 Seats available
1,Delhi to Bareilly (Uttar Pradesh),https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - BLY0076,Ordinary Non AC Seater 2+3,16:30,05h 00m,21:30,3.3,INR 456,52 Seats available
2,Delhi to Bareilly (Uttar Pradesh),https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - RKD0109,Janrath AC Seater 2+2,16:31,05h 29m,22:00,2.9,INR 648,33 Seats available
3,Delhi to Bareilly (Uttar Pradesh),https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - BRT0036,Ordinary Non AC Seater 2+3,17:00,07h 10m,00:10,3.3,INR 493,52 Seats available
4,Delhi to Bareilly (Uttar Pradesh),https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - BLY0148,Rajdhani Non AC Seater 2+3,17:00,05h 00m,22:00,3.3,INR 448,52 Seats available
...,...,...,...,...,...,...,...,...,...,...
1350,Shahjahanpur (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/shahjahanpur...,UPSRTC - LKM0399,Shatabdi AC Seater 2+2,22:15,06h 59m,05:14,3.3,INR 818,35 Seats available
1351,Shahjahanpur (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/shahjahanpur...,UPSRTC - STD0072,Janrath AC Seater 2+2,22:40,07h 05m,05:45,2.5,INR 818,33 Seats available
1352,Shahjahanpur (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/shahjahanpur...,UPSRTC - KSB0395,Janrath AC Seater 2+2,23:05,06h 40m,05:45,3.3,INR 818,35 Seats available
1353,Shahjahanpur (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/shahjahanpur...,UPSRTC - GND0100,Ordinary Non AC Seater 2+3,23:55,08h 05m,08:00,3.3,INR 586,51 Seats available


In [38]:
# WestBengal
# URL of the website
URL = "https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile"
df,driver = main()
# Save the DataFrame to a CSV file
df.to_csv('WestBengal_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/barasat-west-bengal-to-contai-kanthi: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7033D6CB5+28821]
	(No symbol) [0x00007FF703343840]
	(No symbol) [0x00007FF7031E578A]
	(No symbol) [0x00007FF7032391BE]
	(No symbol) [0x00007FF7032394AC]
	(No symbol) [0x00007FF703282647]
	(No symbol) [0x00007FF70325F33F]
	(No symbol) [0x00007FF70327F412]
	(No symbol) [0x00007FF70325F0A3]
	(No symbol) [0x00007FF70322A778]
	(No symbol) [0x00007FF70322B8E1]
	GetHandleVerifier [0x00007FF70370FCAD+3408013]
	GetHandleVerifier [0x00007FF70372741F+3504127]
	GetHandleVerifier [0x00007FF70371B5FD+3455453]
	GetHandleVerifier [0x00007FF70349BDBB+835995]
	(No symbol) [0x00007FF70334EB5F]
	(No symbol) [0x00007FF70334A814]
	(No symbol) [0x00007FF70334A9AD]
	(No symbol) [0x00007FF70333A199]
	BaseThreadInitThunk [0x00007FF8A3B5259D+29]
	RtlUserThreadStart [0x00007FF8A444AF38+40]

Error occurred while scraping bus details for https://www

In [39]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,WBTC (CTC) BARASAT-DIGHA via Bally - 38|15:35,Non AC Seater (2+3),15:35,05h 15m,20:50,3.5,INR 161,39 Seats available
1,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,The Joy Ride,A/C Seater / Sleeper (2+2),21:15,06h 20m,03:35,3.7,INR 650,45 Seats available
2,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,Satya Paribahan,A/C Seater (2+3),23:00,06h 10m,05:10,3.3,INR 320,34 Seats available
3,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,Aradhana Bus Service,Non AC Seater (2+3),16:50,06h 40m,23:30,2.1,INR 350,54 Seats available
4,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,Aradhana Travels,A/C Seater / Sleeper (2+2),18:35,05h 20m,23:55,1.1,450,20 Seats available
...,...,...,...,...,...,...,...,...,...,...
243,Kolkata to Haldia,https://www.redbus.in/bus-tickets/kolkata-to-h...,SBSTC-KOLKATA - HALDIA - 17.30 (HALDIA DEPOT) ...,Non AC Seater (2+3),17:30,03h 00m,20:30,4.2,INR 110,35 Seats available
244,Kolkata to Haldia,https://www.redbus.in/bus-tickets/kolkata-to-h...,SBSTC-KOLKATA - HALDIA - 18.15 (HALDIA DEPOT) ...,Non AC Seater (2+3),18:15,03h 00m,21:15,3.5,INR 110,39 Seats available
245,Kolkata to Haldia,https://www.redbus.in/bus-tickets/kolkata-to-h...,Satadal Travels,A/C Seater (2+2),15:35,02h 40m,18:15,3.8,153,27 Seats available
246,Kolkata to Haldia,https://www.redbus.in/bus-tickets/kolkata-to-h...,Swapna Paribahan,Non AC Seater (2+3),16:55,04h 30m,21:25,3.0,INR 150,25 Seats available
